# Import Packages

In [1]:
import os
import cv2
from google.colab.patches import cv2_imshow
import torch
import imageio

# Supplemental Functions

In [2]:
# create a GIF from the images
def create_gif(image_dir, gif_path, duration = 1):
    images = []

    for filename in os.listdir(image_dir):
        file_path = os.path.join(image_dir, filename)
        images.append(imageio.v2.imread(file_path))

    imageio.mimsave(gif_path, images, duration = duration)

In [3]:
# create a mp4 video from the images
def create_video(image_dir, video_path, fps=1):
    images = []

    for filename in os.listdir(image_dir):
        file_path = os.path.join(image_dir, filename)
        images.append(cv2.imread(file_path))

    height, width, layers = images[0].shape
    video = cv2.VideoWriter(video_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

    for image in images:
        video.write(image)

    video.release()

# Clone Git Repo & Install Dependencies

In [2]:
%cd /content/drive/MyDrive/Gradient Health/Text Detection Project
!git clone https://github.com/meituan/YOLOv6.git
%cd YOLOv6
%pip install -r requirements.txt

/content/drive/MyDrive/Gradient Health/Text Detection Project
fatal: destination path 'YOLOv6' already exists and is not an empty directory.
/content/drive/MyDrive/Gradient Health/Text Detection Project/YOLOv6


# YOLOv6-S

In [3]:
# download the pretrained model
torch.hub.download_url_to_file('https://github.com/meituan/YOLOv6/releases/download/0.4.0/yolov6s.pt', 'yolov6s.pt')

100%|██████████| 38.9M/38.9M [00:00<00:00, 198MB/s]


In [4]:
%%time
# train custom data (single GPU) on train set
# caution: need to modify data/dataset.yaml to only one class ['text']
!python tools/train.py --batch 32 --conf configs/yolov6s_finetune.py --data data/dataset.yaml --fuse_ab --device 0

2023-07-31 16:10:12.138737: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-31 16:10:12.195820: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-31 16:10:13.126141: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Using 1 GPU for training... 
training args are: Namespace(data_path='data/dataset.yaml', conf_file='configs/yolov6s_finetune.py', img_size=640, rect=False, batch_size=32, epochs=400, workers=8, device='0', eval_interval=20, eval_final_only=False, hea

In [5]:
%%time
# evaluation on validation set
# caution: need to modify best_ckpt.pt file path
!python tools/eval.py --data data/dataset.yaml  --weights runs/train/exp1/weights/best_ckpt.pt --task val --device 0

Namespace(data='data/dataset.yaml', weights='runs/train/exp1/weights/best_ckpt.pt', batch_size=32, img_size=640, conf_thres=0.03, iou_thres=0.65, task='val', device='0', half=False, save_dir='runs/val/', name='exp', shrink_size=0, infer_on_rect=True, reproduce_640_eval=False, eval_config_file='./configs/experiment/eval_640_repro.py', do_coco_metric=True, do_pr_metric=False, plot_curve=True, plot_confusion_matrix=False, verbose=False, config_file='', specific_shape=False, height=None, width=None)
Loading checkpoint from runs/train/exp1/weights/best_ckpt.pt

Fusing model...
Switch model to deploy modality.
Model Summary: Params: 18.50M, Gflops: 45.17
img record infomation path is:../custom_dataset/images/.val_cache.json
Val: Checking formats of labels with 8 process(es): 
267 label(s) found, 0 label(s) missing, 0 label(s) empty, 0 invalid label files: 100% 267/267 [00:00<00:00, 1943.60it/s]
Convert to COCO format
100% 267/267 [00:00<00:00, 161156.88it/s]
Convert to COCO format finished. 

In [4]:
%%time
# inference on test set
# caution: need to modify best_ckpt.pt file path
for i in range(1,101):
    filename = '../custom_dataset/images/test/test ' + str(i) + '.jpg'
    !python tools/infer.py --weights runs/train/exp1/weights/best_ckpt.pt --source '{filename}' --yaml data/dataset.yaml --device 0

Namespace(weights='runs/train/exp1/weights/best_ckpt.pt', source='../custom_dataset/images/test/test 1.jpg', webcam=False, webcam_addr='0', yaml='data/dataset.yaml', img_size=[640, 640], conf_thres=0.4, iou_thres=0.45, max_det=1000, device='0', save_txt=False, not_save_img=False, save_dir=None, view_img=False, classes=None, agnostic_nms=False, project='runs/inference', name='exp', hide_labels=False, hide_conf=False, half=False)
Save directory already existed
Loading checkpoint from runs/train/exp1/weights/best_ckpt.pt

Fusing model...
Switch model to deploy modality.
100% 1/1 [00:00<00:00, 19.26it/s]
Results saved to runs/inference/exp
Namespace(weights='runs/train/exp1/weights/best_ckpt.pt', source='../custom_dataset/images/test/test 2.jpg', webcam=False, webcam_addr='0', yaml='data/dataset.yaml', img_size=[640, 640], conf_thres=0.4, iou_thres=0.45, max_det=1000, device='0', save_txt=False, not_save_img=False, save_dir=None, view_img=False, classes=None, agnostic_nms=False, project='r

In [ ]:
# make the images annotated by YOLO to a video for better checking
image_dir = 'runs/inference/exp'
video_path = 'runs/inference/exp/annotation video.mp4'
create_video(image_dir, video_path)

In [3]:
# infer single images
# caution: need to modify best_ckpt.pt file path
!python tools/infer.py --weights runs/train/exp1/weights/best_ckpt.pt --source '../custom_dataset/images/test/infer 1.jpg' --yaml data/dataset.yaml --device 0
!python tools/infer.py --weights runs/train/exp1/weights/best_ckpt.pt --source '../custom_dataset/images/test/infer 2.jpg' --yaml data/dataset.yaml --device 0
# show image
img1 = cv2.imread('runs/inference/exp/infer 1.jpg')
cv2_imshow(img1)
img2 = cv2.imread('runs/inference/exp/infer 2.jpg')
cv2_imshow(img2)

Output hidden; open in https://colab.research.google.com to view.